<a href="https://colab.research.google.com/github/Michael-Jimenez-C/ACM-Material-pygroup/blob/main/ciber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import sympy as sp
from IPython.display import Latex, display_latex
from sympy.solvers.inequalities import reduce_rational_inequalities
import numpy as np
sp.init_session()

IPython console for SymPy 1.7.1 (Python 3.8.15-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.7.1/



#Cayley hamilton

In [ ]:
def cayleyHamilton(M:sp.Matrix,f)->sp.Matrix:
  #Obtiene el polinomio caracteristico
  k=sp.symbols('\lambda')
  ec1=lambda k:sp.det(M-k*sp.eye(M.shape[0]))
  display_latex(Latex(r"P(\lambda)="+sp.latex(ec1(k))))
  #Obtiene las raices
  roots=list(sp.roots(ec1(k)).keys())
  display_latex(Latex("Raices="+sp.latex(roots)))
  #Polinomio residuo
  G=sum([(k)**j*sp.symbols('a_{}'.format(j)) for j in reversed(range(int(sp.degree(ec1(k)))))])-f(k)
  display_latex(Latex(r"r(\lambda)-f(\lambda)="+sp.latex(G)))
  #Construccion del sistema de ecuaciones:
  A=[sp.N(G.subs(k,i)) for i in roots]
  
  if not sp.degree(ec1(k))==len(roots):
    s=G
    while len(A)<sp.degree(ec1(k)):
      s=sp.diff(s,k)
      display_latex(Latex(r"\frac{\partial^n r}{\partial \lambda^n}"+sp.latex(s)))
      A+=[sp.N(s.subs(k,i)) for i in roots]
  A=sp.Matrix(A)
  display_latex(Latex(r"Sistema="+sp.latex(A)))
  #Solución del sistema de ecuaciones
  r=sp.linsolve(A,[sp.symbols('a_{}'.format(j)) for j in reversed(range(int(sp.degree(ec1(k)))))])
  display_latex(Latex("a_n="+sp.latex(r)))
  r=r.args[0][:]
  #Construccion de la funcion matricial
  fx=0
  for i,j in enumerate(reversed(range(0,len(r)-1))):
    fx+=r[j]*(x**(i+1))

  g=lambda s: fx.subs(x, s)+sp.eye(s.shape[0])*r[-1]
  #Aplicacion a la matriz
  _M=(sp.N(sp.simplify(g(sp.Matrix(M)))))
  #Elimina la parte imaginaria si existe
  _M=(_M+_M.conjugate())/2
  return _M#Esta matriz ya está simplificada y transformo algunas expresiones a su forma decimal


In [ ]:
def solucionar(A,B,C,u, X0, t0):
  x,t,T=sp.symbols('x t T', real=True)
  f=lambda x: sp.exp(x*t)
  eAt=cayleyHamilton(A,f).subs(t,t-T)
  eAt=sp.expand(eAt)
  display_latex(Latex("e^{A{(t-\tau)}}="+sp.latex(eAt)))
  
  X=sp.integrate(eAt*B*U,(T,t0,t))
  X+=eAt.subs(T,t0)*X0
  X=sp.simplify(X)
  display_latex(Latex(sp.latex(X)))
  y=C*X
  display_latex(Latex(sp.latex(y)))
  return X,y

Uso

In [ ]:
x,t,T=sp.symbols('x t T', real=True)
f=lambda x: sp.exp(x)
A=[[5,6,5,7,5,5],
   [0,1,7,2,3,7],
   [0,0,2,5,2,2],
   [0,0,0,1,5,9],
   [0,0,0,1,6,1],
   [0,0,9,5,7,1]]
A=sp.Matrix(A)
k=cayleyHamilton(A,f)
k

Matrix([
[148.413159102461, 218.542315911003, 336524.732028032, 377954.758988171,  781409.90771335, 407686.066228491],
[               0, 2.71828182845957, 137953.865347805, 155050.701545433, 321118.700146976,   167248.2790696],
[               0,                0, 74237.8227366377, 83446.1730777676, 172870.320717254, 90012.0329544519],
[               0,                0, 102238.324593814, 114926.872747264, 238147.371353801,   123971.5939589],
[               0,                0, 31785.5902676427, 35744.7041240955, 74189.8067473831, 38561.4924674711],
[               0,                0, 117672.746542855,  132274.82705521,  274077.77707426, 142684.268103273]])

In [ ]:
x,t,T=sp.symbols('x t T', real=True)
f=lambda x: sp.exp(x)
A=[[1,4,7],[0,3,0],[0,5,2]]
A=sp.Matrix(A)
k=cayleyHamilton(A,f)
k

Matrix([
[2.71828182845905, 175.184374880702, 32.6954198933012],
[               0, 20.0855369231877,                0],
[               0, 63.4824041212851, 7.38905609893065]])

In [ ]:
x,t,T=sp.symbols('x t T', real=True)
f=lambda x: sp.sin(x)
A=[[1,4,7],[0,3,0],[0,5,2]]
A=sp.Matrix(A)
k=cayleyHamilton(A,f)
k

Matrix([
[0.841470984807896, -16.0307695172091, 0.474785094124496],
[                0, 0.141120008059867,                 0],
[                0, -3.84088709382907, 0.909297426825681]])

Solucion de ecuacion de estados

In [ ]:
A=sp.Matrix([[-3,1],[0,-2]])

B=sp.Matrix([[0],[1]])

C=sp.Matrix([[1,1]])

u=sp.symbols('u')
U=sp.Matrix([[0]])
X0=sp.Matrix([[1],[-1]])
t0=0
solucionar(A,B,C,U,X0,t0)

(Matrix([
 [(2.0 - 1.0*exp(t))*exp(-3*t)],
 [              -1.0*exp(-2*t)]]),
 Matrix([[(2.0 - 1.0*exp(t))*exp(-3*t) - 1.0*exp(-2*t)]]))

In [ ]:
A=sp.Matrix([[-3,1],[0,-2]])

B=sp.Matrix([[1/2],
             [0]])

C=sp.Matrix([[1,0]])

u=sp.symbols('u')
U=sp.Matrix([[u]])
X0=sp.Matrix([[1],[-1]])
t0=0
sol=solucionar(A,B,C,U,X0,t0)
sol

(Matrix([
 [(0.166666666666667*u*exp(3*t) - 0.166666666666667*u - 1.0*exp(t) + 2.0)*exp(-3*t)],
 [                                                                   -1.0*exp(-2*t)]]),
 Matrix([[(0.166666666666667*u*exp(3*t) - 0.166666666666667*u - 1.0*exp(t) + 2.0)*exp(-3*t)]]))

In [ ]:
A=sp.Matrix([[0,1],[-2,-2]])

B=sp.Matrix([[0],
             [1]])

C=sp.Matrix([[1,1]])

u=sp.symbols('u')
U=sp.Matrix([[1]])
X0=sp.Matrix([[1],[0]])
t0=0
sol=solucionar(A,B,C,U,X0,t0)[0]
sol

Matrix([
[(0.25*(1 - I)*exp(2*I*t) + 0.5*exp(t*(1 + I)) + 0.25 + 0.25*I)*exp(-t*(1 + I))],
[                                        0.5*I*(exp(2*I*t) - 1)*exp(-t*(1 + I))]])

In [ ]:
sp.expand(sol)

Matrix([
[0.5 + 0.25*exp(-t)*exp(I*t) - 0.25*I*exp(-t)*exp(I*t) + 0.25*exp(-t)*exp(-I*t) + 0.25*I*exp(-t)*exp(-I*t)],
[                                                         0.5*I*exp(-t)*exp(I*t) - 0.5*I*exp(-t)*exp(-I*t)]])

In [ ]:
sp.N(sol.subs(t,1))

Matrix([
[0.754162992999763 + 2.71050543121376e-20*I],
[-0.309559875653112 - 3.3881317890172e-21*I]])

In [ ]:
A=sp.Matrix([[0,1],[-6,-5]])

B=sp.Matrix([[0],
             [1]])

C=sp.Matrix([[1,1]])

u=sp.symbols('u')
U=sp.Matrix([[1]])
X0=sp.Matrix([[1],[0]])
t0=0
sol=solucionar(A,B,C,U,X0,t0)[0]
sol

Matrix([
[0.166666666666667 + 2.5*exp(-2*t) - 1.66666666666667*exp(-3*t)],
[                                    5.0*(1 - exp(t))*exp(-3*t)]])

In [ ]:
sp.expand(sol)

Matrix([
[0.166666666666667 + 2.5*exp(-2*t) - 1.66666666666667*exp(-3*t)],
[                                -5.0*exp(-2*t) + 5.0*exp(-3*t)]])

In [ ]:
A=sp.Matrix([[0,1],[-1,-2]])

B=sp.Matrix([[1],
             [1]])

C=sp.Matrix([[1,1]])

u=sp.symbols('u')
U=sp.Matrix([[1]])
X0=sp.Matrix([[1],[0]])
t0=0
sol=solucionar(A,B,C,U,X0,t0)[0]
sol

(Matrix([
 [(-1.0*t + 3.0*exp(t) - 2.0)*exp(-t)],
 [       1.0*(t - exp(t) + 1)*exp(-t)]]),
 Matrix([[(-1.0*t + 3.0*exp(t) - 2.0)*exp(-t) + 1.0*(t - exp(t) + 1)*exp(-t)]]))

In [ ]:
sp.expand(sol)

Matrix([
[-1.0*t*exp(-t) + 3.0 - 2.0*exp(-t)],
[ 1.0*t*exp(-t) - 1.0 + 1.0*exp(-t)]])

In [ ]:
A=sp.Matrix([[0,1],[-2,-3]])

B=sp.Matrix([[-1],
             [1]])
                 
C=sp.Matrix([[0,1]])

u=sp.symbols('u')
U=sp.Matrix([[1]])
X0=sp.Matrix([[1],[-1]])
t0=0
sol=solucionar(A,B,C,U,X0,t0)
sol

(Matrix([
 [-1.0 + 2.0*exp(-t)],
 [ 1.0 - 2.0*exp(-t)]]), Matrix([[1.0 - 2.0*exp(-t)]]))

#Linealización

In [ ]:
def linealizarx(f:list,h:list,varx:list,varu:list)->sp.Matrix:
  def deriv(f,var):
    p0=sp.symbols('p0')
    A=[]
    for i in range(len(f)):
      A.append([])
      for j in var:
        A[-1].append(sp.diff(f[i],j).subs(j,p0))
    return sp.Matrix(A)
  A=deriv(f,varx)
  display_latex(Latex(r"A="+sp.latex(A)))
  B=deriv(f,varu)
  display_latex(Latex(r"B="+sp.latex(B)))
  C=deriv(h,varx)
  display_latex(Latex(r"C="+sp.latex(C)))
  D=deriv(h,varu)
  display_latex(Latex(r"D="+sp.latex(D)))
  return A,B,C,D


In [ ]:
x1,x2,ta=sp.symbols('x1 x2 ta')

f=[x2,
   4*x1-x1**3-sp.sqrt(x2**3)+ta]
h=[x2]
A,B,C,D=linealizarx(f,h,[x1,x2],[ta])

#transformación de similitud

In [ ]:
def Tsim(A :sp.Matrix,B : sp.Matrix):
  k=sp.symbols('\lambda')
  poli=sp.det(A-k*sp.eye(A.shape[0]))
  display_latex(Latex(r"p(\lambda)="+sp.latex(poli)))
  coeficientes=[poli.coeff(k**i) for i in range(1,sp.degree(poli))]+[1]
  P=sp.zeros(sp.degree(poli))
  for i in range(sp.degree(poli)):
    for j in range(sp.degree(poli)-i):
      P[i,j]=coeficientes[j+i]
  display_latex(Latex(r"P="+sp.latex(P)))
  C0=[]
  for i in range(sp.degree(poli)):
    C0.append(list((A**i)*B))
  C0=sp.Matrix(C0).T
  display_latex(Latex(r"C_0="+sp.latex(C0)))
  
  M=C0*P

  display_latex(Latex(r"M="+sp.latex(M)))
  return P,C0,M


In [ ]:
EstadosAFTransf=lambda A,B,C:C*(sp.symbols('s')*sp.eye(2)-A).inv()*B

In [ ]:
A=sp.Matrix([[1,2,1,1],[3,1,-1,3],[2,1,1,2],[-1,0,0,3]])
B=sp.Matrix([[0],[2],[1],[1]])
Tsim(A,B)

(Matrix([
 [25,  6, -6, 1],
 [ 6, -6,  1, 0],
 [-6,  1,  0, 0],
 [ 1,  0,  0, 0]]), Matrix([
 [0, 6, 22, 104],
 [2, 4, 26,  74],
 [1, 5, 27, 103],
 [1, 3,  3, -13]]), Matrix([
 [ 8, -14,  6, 0],
 [-8,  14, -8, 2],
 [-4,   3, -1, 1],
 [12,  -9, -3, 1]]))

In [ ]:
def TCanObs(A :sp.Matrix,C : sp.Matrix):
  k=sp.symbols('\lambda')
  poli=sp.det(A-k*sp.eye(A.shape[0]))
  display_latex(Latex(r"p(\lambda)="+sp.latex(poli)))
  coeficientes=[poli.coeff(k**i) for i in range(1,sp.degree(poli))]+[1]
  P=sp.zeros(sp.degree(poli))
  for i in range(sp.degree(poli)):
    for j in range(sp.degree(poli)-i):
      P[i,-j-1]=coeficientes[j+i]
  P=P.T
  display_latex(Latex(r"P="+sp.latex(P)))
  Ob=[]
  for i in range(sp.degree(poli)):
    Ob.append(list(C*(A**i)))
  Ob=sp.Matrix(Ob)

  display_latex(Latex(r"O_b="+sp.latex(Ob)))
  M=P*Ob
  display_latex(Latex(r"P*O_b="+sp.latex(M)))
  M=(M).inv()

  display_latex(Latex(r"M="+sp.latex(M)))
  return P,Ob,M

In [ ]:
C=sp.Matrix([[1,-1,2,0]])
TCanObs(A,C)

(Matrix([
 [ 1,  0,  0, 0],
 [-6,  1,  0, 0],
 [ 6, -6,  1, 0],
 [25,  6, -6, 1]]), Matrix([
 [ 1, -1, 2,   0],
 [ 2,  3, 4,   2],
 [17, 11, 3,  25],
 [31, 48, 9, 131]]), Matrix([
 [   117/235,    -1/235,    -2/235,   -4/235],
 [ 3433/5875,  756/5875, -133/5875, -31/5875],
 [6383/11750, 781/11750, -83/11750, 69/11750],
 [ 1267/2350,  419/2350,  133/2350,  31/2350]]))

#Test de routh

In [95]:
def routhTest(polinomio,K=sp.symbols('k',real=True)):
  s,e=sp.symbols('s \epsilon')
  '''
  try:
    r=[sp.N(i) for i in sp.roots(polinomio)]
    raices=[(i+i.conjugate())/2 for i in r]
    display_latex(Latex("s_i="+sp.latex(raices)))
    raices=np.array(raices)
    if len(raices)>len(raices[raices<0]):
      return False
  except:
    pass
  '''
  grado=sp.degree(polinomio,s)


  display_latex(Latex("deg(p(s))="+str(grado)))


  coeficientes=[polinomio.coeff(s**i) for i in reversed(range(1,grado+1))]+[polinomio.subs(s,0)]
  tabla=sp.zeros(grado+1)
  tabla[0,:len(range(0,len(coeficientes),2))]=sp.Matrix([[coeficientes[i] for i in range(0,len(coeficientes),2)]])
  tabla[1,:len(range(1,len(coeficientes),2))]=sp.Matrix([[coeficientes[i] for i in range(1,len(coeficientes),2)]])
  Ki=None
  for i in range(1,grado):
    if Ki==None:
      Ki=[tabla[0,0]/tabla[1,0]]
    else:
      Ki.append(tabla[i-1,0]/(tabla[i,0] if tabla[i,0]!=0 else e))
    temp=list(tabla[i-1,:]-Ki[-1]*tabla[i,:])
    temp.pop(0)
    temp.append(0)
    tabla[i+1,:]=sp.Matrix([temp])
  tabla=sp.expand(tabla)
  tabla=sp.simplify(tabla)

  temp=r""
  for i in range(len(Ki)):
    temp+=r"k_{}=& {}\\".format(grado-1-i,sp.latex(sp.simplify(Ki[i])))

  display_latex(Latex(r"\begin{cases}"+temp+r"\end{cases}"))

  del(temp)

  display_latex(Latex(sp.latex(tabla)))


  condiciones=[sp.solve(i>0) for i in tabla[:,0]]

  return condiciones
  try:
    s=None
    for i in condiciones:
      print("E", i)
      if s==None:
        if type(i)!=list:
          print(i)
          s=i
        continue
      s=s and i
      print("s",s)

    print(v)
  except:
    v=True
    for i in condiciones:
      v= v and i

  if v==True:
    return "Es hurtwiz"
  elif v==False:
    return "No es hurtwiz"


In [71]:
s=sp.symbols('s')
p=2*s**4+s**3+5*s**2+2*s+1
#p=2*s**4+s**3+5*s**2+3*s+4 
p

In [72]:
routhTest(p)

[[], [], [], [], []]


Probar 
$$s^3+(5+k)s^2+(12-2k)s+5k-18$$

In [96]:
s,k=sp.symbols('s k')
p=s**3+(5+k)*s**2+(12-2*k)*s+5*k-18 
p

In [104]:
M=routhTest(p)


In [105]:
M=M[1:]

[(-5 < k) & (k < oo), ((-oo < k) & (k < -sqrt(633)/4 - 3/4)) | ((-5 < k) & (k < -3/4 + sqrt(633)/4)), (18/5 < k) & (k < oo)]


In [109]:
M

AttributeError: ignored